In [1]:
import numpy as np
import networkx as nx
import time
import pandas as pd

def create_graph(cities_count, complete=True, threshold=0.5):
    mat = np.random.random((cities_count, cities_count))
    mat = mat + mat.T
    np.fill_diagonal(mat, 0)
    adj_matrix = mat * cities_count
    if not complete:
        adj_matrix[adj_matrix < threshold] = 0
    G = nx.from_numpy_array(adj_matrix)
    if not nx.is_connected(G):
        raise ValueError("Le graphe généré n'est pas connecté. Essayez de régénérer la matrice.")
    return G, adj_matrix

def gen_path(start, pheromone, distances, alpha, beta):
    path = []
    visited = set()
    visited.add(start)
    prev = start
    for _ in range(len(distances) - 1):
        move = pick_move(pheromone[prev, :], distances[prev, :], visited, alpha, beta)
        path.append((prev, move))
        prev = move
        visited.add(move)
    path.append((prev, start))
    return path

def pick_move(pheromone_row, dist_row, visited, alpha, beta):
    pheromone_row = np.copy(pheromone_row)
    pheromone_row[list(visited)] = 0
    dist_row[dist_row == 0] = np.inf
    with np.errstate(divide='ignore'):
        row = pheromone_row ** alpha * ((1.0 / dist_row) ** beta)
    row = np.nan_to_num(row)
    norm_row = row / row.sum()
    move = np.random.choice(range(len(pheromone_row)), 1, p=norm_row)[0]
    return move

def gen_all_paths(n_ants, pheromone, distances, alpha, beta):
    all_paths = []
    for i in range(n_ants):
        path = gen_path(0, pheromone, distances, alpha, beta)
        all_paths.append((path, calc_path_length(path, distances)))
    return all_paths

def calc_path_length(path, distances):
    total_dist = 0
    for (i, j) in path:
        total_dist += distances[i, j]
    return total_dist

def spread_pheromone(all_paths, pheromone, n_best, decay):
    sorted_paths = sorted(all_paths, key=lambda x: x[1])
    for path, dist in sorted_paths[:n_best]:
        for move in path:
            pheromone[move] += 1.0 / dist
    pheromone *= decay

def run_aco(n_ants, n_best, n_iterations, decay, alpha, beta, adj_matrix):
    pheromone = np.ones(adj_matrix.shape) / len(adj_matrix)
    all_time_shortest_path = ("placeholder", np.inf)
    path_lengths = []
    for i in range(n_iterations):
        all_paths = gen_all_paths(n_ants, pheromone, adj_matrix, alpha, beta)
        spread_pheromone(all_paths, pheromone, n_best, decay)
        shortest_path = min(all_paths, key=lambda x: x[1])
        path_lengths.append(shortest_path[1])
        if shortest_path[1] < all_time_shortest_path[1]:
            all_time_shortest_path = shortest_path
    return all_time_shortest_path[1], path_lengths

# Collect data
cities_count = 30
param_grid = {
    'n_ants': [10, 20, 30, 40],
    'n_best': [2, 5, 10, 15],
    'n_iterations': [25, 50, 75, 100],
    'decay': [0.3, 0.5, 0.7, 0.9],
    'alpha': [1, 1.5, 2, 2.5],
    'beta': [2, 3, 4, 5]
}

results = []

for n_ants in param_grid['n_ants']:
    for n_best in param_grid['n_best']:
        for n_iterations in param_grid['n_iterations']:
            for decay in param_grid['decay']:
                for alpha in param_grid['alpha']:
                    for beta in param_grid['beta']:
                        for _ in range(3):  # Repeat 3 times for statistical significance
                            G, adj_matrix = create_graph(cities_count, complete=True)
                            distance, path_lengths = run_aco(n_ants, n_best, n_iterations, decay, alpha, beta, adj_matrix)
                            results.append({
                                'n_ants': n_ants,
                                'n_best': n_best,
                                'n_iterations': n_iterations,
                                'decay': decay,
                                'alpha': alpha,
                                'beta': beta,
                                'distance': distance,
                                'iterations_needed': next((i for i, length in enumerate(path_lengths) if length == distance), n_iterations)
                            })

df_results = pd.DataFrame(results)
df_results.to_csv('aco_results.csv', index=False)


KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Charger les données
df = pd.read_csv('aco_results.csv')

# Définir les variables indépendantes et dépendantes
X = df[['n_ants', 'n_best', 'n_iterations', 'decay', 'alpha', 'beta']]
y = df['distance']

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Créer et entraîner le modèle de régression linéaire
model = LinearRegression()
model.fit(X_train, y_train)

# Prédire les valeurs et évaluer le modèle
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Afficher les coefficients du modèle
coefficients = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])
print(coefficients)


In [ ]:
from bayes_opt import BayesianOptimization

def aco_objective(n_ants, n_best, n_iterations, decay, alpha, beta):
    n_ants = int(n_ants)
    n_best = int(n_best)
    n_iterations = int(n_iterations)
    G, adj_matrix = create_graph(cities_count, complete=True)
    distance, _ = run_aco(n_ants, n_best, n_iterations, decay, alpha, beta, adj_matrix)
    return -distance  # Maximiser l'opposé de la distance pour minimiser la distance

# Définir les limites pour chaque paramètre
pbounds = {
    'n_ants': (10, 50),
    'n_best': (2, 20),
    'n_iterations': (25, 100),
    'decay': (0.3, 0.9),
    'alpha': (1, 2.5),
    'beta': (2, 5)
}

# Initialiser l'optimiseur bayésien
optimizer = BayesianOptimization(
    f=aco_objective,
    pbounds=pbounds,
    random_state=42
)

# Lancer le processus d'optimisation
optimizer.maximize(
    init_points=10,
    n_iter=50
)

# Afficher les paramètres optimaux
print(optimizer.max)
